In [1]:
import os
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
POSTGRES = {

'user' : 'postgres',
'pw' : 'admin',
'host' : 'localhost',
'port' : '5432',
'db' : 'soccerdata'
}

In [3]:
class Config(object):
    # ...
    SQLALCHEMY_DATABASE_URI =  os.environ.get('DATABASE_URL') or 'postgresql://%(user)s:%(pw)s@%(host)s:%(port)s/%(db)s' % POSTGRES
    SQLALCHEMY_TRACK_MODIFICATIONS = False

config = Config()

cnx = create_engine(config.SQLALCHEMY_DATABASE_URI)

In [73]:
df = pd.read_sql_query('''
SELECT f.*, l.event_date, l.home_goals, l.away_goals, l.home_team, l.home_team_id, l.away_team, l.away_team_id 
FROM fixture f left join league l on l.fixture_id = f.fixture_id;''', cnx)

df.rename(columns={'home_goals': 'goals_home', 'away_goals': 'goals_away'}, inplace=True)

conditions = [(df['goals_away'] < df['goals_home']), (df['goals_away'] == df['goals_home']), (df['goals_away'] > df['goals_home'])]
values_home = ['W', 'D', 'L']
values_away = ['L', 'D', 'W']

df['home_wld'] = np.select(conditions, values_home)
df['away_wld'] = np.select(conditions, values_away)

df = df.sort_values('event_date')
df.reset_index(inplace=True, drop=True)

In [5]:
df

,fixture_id,league_id,shots_on_goal_home,shots_on_goal_away,shots_off_goal_home,shots_off_goal_away,total_shots_home,total_shots_away,blocked_shots_home,blocked_shots_away,...,passes_perc_away,event_date,goals_home,goals_away,home_team,home_team_id,away_team,away_team_id,home_wld,away_wld
0,192875,697,6,3,3,8,0,0,0,0,...,0.00,2015-01-01,1,1,West Ham,48,West Brom,60,D,D
1,192867,697,3,2,5,3,0,0,0,0,...,0.00,2015-01-01,1,1,Stoke City,75,Manchester United,33,D,D
2,192868,697,3,2,7,3,0,0,0,0,...,0.00,2015-01-01,0,0,Aston Villa,66,Crystal Palace,52,D,D
3,192869,697,5,3,5,5,0,0,0,0,...,0.00,2015-01-01,2,0,Hull City,64,Everton,45,W,L
4,192870,697,6,4,9,7,0,0,0,0,...,0.00,2015-01-01,2,2,Liverpool,40,Leicester,46,D,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288,157026,524,7,4,9,4,21,12,5,4,...,0.71,2019-08-17,1,2,Aston Villa,66,Bournemouth,35,L,W
1289,157033,524,3,6,7,6,14,15,4,3,...,0.78,2019-08-17,1,2,Southampton,41,Liverpool,40,L,W
1290,157031,524,8,4,3,7,15,12,4,1,...,0.81,2019-08-17,3,1,Norwich,71,Newcastle,34,W,L
1291,157025,524,9,5,3,9,15,18,3,4,...,0.65,2019-08-17,2,1,Arsenal,42,Burnley,44,W,L


### Features Engineering

#### Utils

In [18]:
def get_goals_scored(df, team):
    mask_home = np.where(df['home_team'] == team)
    mask_away = np.where(df['away_team'] == team)
    
    goals_scored_home = df.loc[mask_home[0], ['event_date','goals_home']].rename(columns={'goals_home': 'goals_scored'})
    goals_scored_away = df.loc[mask_away[0], ['event_date','goals_away']].rename(columns={'goals_away': 'goals_scored'})

    goals_scored = pd.concat([goals_scored_home, goals_scored_away]).sort_values(by='event_date')

    return goals_scored
    
def get_goals_taken(df, team):
    
    mask_home = np.where(df['home_team'] == team)
    mask_away = np.where(df['away_team'] == team)
    
    goals_taken_away = df.loc[mask_away[0],  ['event_date','goals_home']].rename(columns={'goals_home': 'goals_taken'})
    goals_taken_home = df.loc[mask_home[0],  ['event_date','goals_away']].rename(columns={'goals_away': 'goals_taken'})

    goals_taken = pd.concat([goals_taken_home, goals_taken_away]).sort_values(by='event_date')
    
    return goals_taken

def expanding_average_goals(df):
    
    df.loc[:, ['average_goals_scored_home', 'average_goals_taken_home', 'average_goals_scored_away', 
               'average_goals_taken_away']] = 0
    
    for team in np.unique(list(df['home_team'].values) + list(df['away_team'].values)):
        
        team_mask = np.where((df['home_team'] == team) | (df['away_team'] == team))
        df_team = df.loc[team_mask[0], :]
        df_team.reset_index(inplace=True, drop=False)

        res_taken = get_goals_taken(df_team, team)
        res_taken = res_taken['goals_taken'].expanding(1).mean().shift()

        res_scored = get_goals_scored(df_team, team)
        res_scored = res_scored['goals_scored'].expanding(1).mean().shift()

        df_team.loc[np.where(df_team['home_team'] == team)[0], 'is_home'] = True
        df_team.loc[np.where(df_team['home_team'] != team)[0], 'is_home'] = False
        df_team.loc[np.where(df_team['away_team'] == team)[0], 'is_away'] = True
        df_team.loc[np.where(df_team['away_team'] != team)[0], 'is_away'] = False

        df_team.loc[df_team['is_home'].values,'average_goals_scored_home'] = res_scored[df_team['is_home'].values].values
        df_team.loc[df_team['is_home'].values,'average_goals_taken_home'] = res_taken[df_team['is_home'].values].values

        df_team.loc[df_team['is_away'].values,'average_goals_scored_away'] = res_scored[df_team['is_away'].values].values
        df_team.loc[df_team['is_away'].values,'average_goals_taken_away'] = res_taken[df_team['is_away'].values].values

        df.loc[df_team['index'], 'average_goals_scored_home'] = df_team.loc[:, 'average_goals_scored_home'].values
        df.loc[df_team['index'], 'average_goals_taken_home'] = df_team.loc[:, 'average_goals_taken_home'].values
        df.loc[df_team['index'], 'average_goals_scored_away'] = df_team.loc[:, 'average_goals_scored_away'].values
        df.loc[df_team['index'], 'average_goals_taken_away'] = df_team.loc[:, 'average_goals_taken_away'].values
        
    return df

def get_stat(df, team, stat):
    mask_home = np.where(df['home_team'] == team)
    mask_away = np.where(df['away_team'] == team)
    
    if ('_home' in stat) or ('_away' in stat):
        stat_home = df.loc[df.index.intersection(mask_home[0]), stat]
        stat_away = df.loc[df.index.intersection(mask_away[0]), stat]
    else:
        stat_home = df.loc[df.index.intersection(mask_home[0]), stat + '_home']
        stat_away = df.loc[df.index.intersection(mask_away[0]), stat + '_away']

    stat_team = pd.concat([stat_home, stat_away])
    
    return stat_team

def expanding_average_stat(df, stat):
    
    for team in np.unique(list(df['home_team'].values) + list(df['away_team'].values)):
        team_mask = np.where((df['home_team'] == team) | (df['away_team'] == team))
        df_team = df.loc[team_mask[0], :]
        df_team.reset_index(inplace=True, drop=False)

        res, res_home, res_away = pd.Series(get_stat(df_team, team, stat))
        res = res.expanding(1).mean().shift()
        # res_home = res.expanding(1).mean().shift()
        # res_away = res_away.expanding(1).mean().shift()

        df_team.loc[np.where(df_team['home_team'] == team)[0], 'is_home'] = True
        df_team.loc[np.where(df_team['home_team'] != team)[0], 'is_home'] = False
        df_team.loc[np.where(df_team['away_team'] == team)[0], 'is_away'] = True
        df_team.loc[np.where(df_team['away_team'] != team)[0], 'is_away'] = False

        if ('_home' not in stat) and ('_away' not in stat):
            home_stat_label = stat + '_home'
            away_stat_label = stat + '_away'
        else:
            home_stat_label = stat
            away_stat_label = stat
        
        df_team.loc[df_team['is_home'].values, 'overall_average_' + home_stat_label] = res[df_team['is_home'].values].values
        df_team.loc[df_team['is_away'].values, 'overall_average_' + away_stat_label] = res[df_team['is_away'].values].values

        df_team.loc[df_team['is_home'].values, 'home_average_' + home_stat_label] = res[df_team['is_home'].values].values
        df_team.loc[df_team['is_away'].values, 'away_average_' + away_stat_label] = res[df_team['is_away'].values].values
        
        df_team.loc[df_team['is_home'].values, 'overall_average_' + home_stat_label] = res[df_team['is_home'].values].values
        df_team.loc[df_team['is_away'].values, 'overall_average_' + away_stat_label] = res[df_team['is_away'].values].values

        df.loc[df_team['index'], 'overall_average_' + home_stat_label] = df_team.loc[:, 'overall_average_' + home_stat_label].values
        df.loc[df_team['index'], 'overall_average_' + away_stat_label] = df_team.loc[:, 'overall_average_' + away_stat_label].values
        
    return df

def get_stat(df, team, stat):
    mask_home = np.where(df['home_team'] == team)
    mask_away = np.where(df['away_team'] == team)
    
#     if ('_home' in stat) or ('_away' in stat):
    stat_home = df[stat].reindex(index=mask_home[0])
    stat_away = df[stat].reindex(index=mask_away[0])
#     else:
#         stat_home = df.loc[mask_home[0], stat + '_home']
#         stat_away = df.loc[mask_away[0], stat + '_away']

    stat_team = pd.concat([stat_home, stat_away])
    
    return stat_team, stat_home, stat_away

#### Apply to all relevant metrics

In [74]:
cols = ['shots_on_goal_home', 'shots_on_goal_away','shots_off_goal_home', 'shots_off_goal_away', 'total_shots_home',
        'total_shots_away', 'blocked_shots_home', 'blocked_shots_away','shots_insidebox_home', 'shots_insidebox_away', 
        'shots_outsidebox_home', 'shots_outsidebox_away', 'fouls_home', 'fouls_away', 'corner_kicks_home', 'corner_kicks_away', 
        'offsides_home', 'offsides_away', 'ball_possession_home', 'ball_possession_away', 'yellow_cards_home', 
        'yellow_cards_away', 'red_cards_home', 'red_cards_away', 'goalkeeper_saves_home', 'goalkeeper_saves_away', 
        'total_passes_home', 'total_passes_away', 'passes_accurate_home', 'passes_accurate_away', 'passes_perc_home', 
        'passes_perc_away']

df = expanding_average_goals(df)

# for col in cols:
#     df = expanding_average_stat(df, col)
    
# df = df.dropna(axis=0, how='any')

In [68]:
df.loc[(df['home_team']=='Crystal Palace') | (df['away_team']=='Crystal Palace'),['event_date','home_team','away_team',
                                                                                  'goals_home','goals_away',
                                                                                  'average_goals_taken_home',
                                                                                  'average_goals_taken_away']]

,event_date,home_team,away_team,goals_home,goals_away,average_goals_taken_home,average_goals_taken_away
10,2015-01-10,Crystal Palace,Tottenham,2,1,0.000000,3.000000
20,2015-01-17,Burnley,Crystal Palace,2,3,2.000000,0.500000
33,2015-01-31,Crystal Palace,Everton,0,1,1.000000,1.000000
42,2015-02-07,Leicester,Crystal Palace,0,1,1.500000,1.000000
56,2015-02-11,Crystal Palace,Newcastle,1,1,0.800000,1.600000
...,...,...,...,...,...,...,...
1218,2018-05-05,Stoke City,Crystal Palace,1,2,1.540984,1.459016
1230,2018-05-13,Crystal Palace,West Brom,2,0,1.455285,1.308943
1257,2018-08-20,Crystal Palace,Liverpool,0,2,1.432000,1.064516
1266,2018-08-26,Watford,Crystal Palace,2,1,1.577586,1.436508


In [52]:
team = 'Crystal Palace'

team_mask = (df['home_team'] == team) | (df['away_team'] == team)
df_team = df.loc[team_mask[0], :]
df_team.reset_index(inplace=True, drop=False)

res_taken = get_goals_taken(df_team, team)
res_taken = res_taken['goals_taken'].expanding(1).mean().shift()

res_scored = get_goals_scored(df_team, team)
res_scored = res_scored['goals_scored'].expanding(1).mean().shift()

1.6666666666666667

In [70]:
team = 'Crystal Palace'

team_mask = (df['home_team'] == team) | (df['away_team'] == team)
df_team = df.loc[team_mask, :]
df_team.reset_index(inplace=True, drop=False)

res_taken = get_goals_taken(df_team, team)

In [71]:
res_taken['goals_taken'].expanding(1).mean().shift()

0           NaN
1      1.000000
2      1.500000
3      1.333333
4      1.000000
         ...   
119    1.462185
120    1.458333
121    1.446281
122    1.450820
123    1.455285
Name: goals_taken, Length: 124, dtype: float64

In [63]:
res_taken

,event_date,goals_taken
0,2015-01-10,1
1,2015-01-17,2
2,2015-01-31,1
3,2015-02-07,0
4,2015-02-11,1
...,...,...
119,2018-05-05,1
120,2018-05-13,0
121,2018-08-20,2
122,2018-08-26,2


In [76]:
team_mask = (df['home_team'] == team) | (df['away_team'] == team)

df.loc[team_mask,['event_date','home_team','away_team','goals_home','goals_away','average_goals_taken_home',
                  'average_goals_taken_away']]

,event_date,home_team,away_team,goals_home,goals_away,average_goals_taken_home,average_goals_taken_away
2,2015-01-01,Aston Villa,Crystal Palace,0,0,NaN,NaN
10,2015-01-10,Crystal Palace,Tottenham,2,1,0.000000,3.000000
20,2015-01-17,Burnley,Crystal Palace,2,3,2.000000,0.500000
33,2015-01-31,Crystal Palace,Everton,0,1,1.000000,1.000000
42,2015-02-07,Leicester,Crystal Palace,0,1,1.500000,1.000000
...,...,...,...,...,...,...,...
1230,2018-05-13,Crystal Palace,West Brom,2,0,1.455285,1.308943
1241,2018-08-11,Fulham,Crystal Palace,0,2,NaN,1.443548
1257,2018-08-20,Crystal Palace,Liverpool,0,2,1.432000,1.064516
1266,2018-08-26,Watford,Crystal Palace,2,1,1.577586,1.436508


### Classification

In [10]:
features = ['overall_average_goals_scored_home', 'overall_average_goals_taken_home', 'overall_average_goals_scored_away',
            'overall_average_goals_taken_away', 'overall_average_shots_on_goal_home', 'overall_average_shots_on_goal_away',
            'overall_average_shots_off_goal_home', 'overall_average_shots_off_goal_away', 'overall_average_total_shots_home', 
            'overall_average_total_shots_away', 'overall_average_blocked_shots_home', 
        'average_blocked_shots_away', 'average_shots_insidebox_home', 'average_shots_insidebox_away', 
        'average_shots_outsidebox_home', 'average_shots_outsidebox_away', 'average_fouls_home', 'average_fouls_away', 
        'average_corner_kicks_home', 'average_corner_kicks_away', 'average_offsides_home', 'average_offsides_away', 
        'average_ball_possession_home', 'average_ball_possession_away', 'average_yellow_cards_home', 
        'average_yellow_cards_away', 'average_red_cards_home', 'average_red_cards_away', 'average_goalkeeper_saves_home', 
        'average_goalkeeper_saves_away', 'average_total_passes_home', 'average_total_passes_away', 
        'average_passes_accurate_home', 'average_passes_accurate_away', 'average_passes_perc_home', 'average_passes_perc_away']

df['home_wld'] = LabelEncoder().fit_transform(df['home_wld'])
X = df[features + ['home_wld']]
y = X['home_wld']
X = df[features]

n = X.shape[0]
idx_train = int(np.round(2*n/3,0))
y_train = y[:idx_train]
y_test = y[idx_train:]

X_train = X.iloc[:idx_train,:]
X_test = X.iloc[idx_train:,:]

KeyError: "['average_passes_accurate_away', 'overall_average_goals_taken_away', 'average_corner_kicks_away', 'average_total_passes_home', 'average_offsides_home', 'average_ball_possession_away', 'average_red_cards_away', 'average_ball_possession_home', 'average_corner_kicks_home', 'average_passes_perc_away', 'average_goalkeeper_saves_away', 'average_offsides_away', 'average_passes_perc_home', 'overall_average_goals_scored_home', 'overall_average_goals_taken_home', 'average_shots_insidebox_home', 'average_shots_outsidebox_home', 'average_yellow_cards_home', 'average_fouls_away', 'average_total_passes_away', 'average_goalkeeper_saves_home', 'average_fouls_home', 'average_red_cards_home', 'overall_average_goals_scored_away', 'average_yellow_cards_away', 'average_passes_accurate_home', 'average_blocked_shots_away', 'average_shots_outsidebox_away', 'average_shots_insidebox_away'] not in index"

In [14]:
df['ex']

KeyError: 'average_passes_accurate_home'